In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

/home/jac/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


file path ../features/cnn_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_glove_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_glove_1_trainable_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat_de_fr.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
fi

In [2]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((226998,6))
    cache_test_pred = np.zeros((226998,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'colsample_bytree': 0.9,
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=2000)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((226998,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [3]:
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold5.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

# sample_submission = pd.read_csv("../input/sample_submission.csv")
# sample_submission[list_classes] = b
# sample_submission.to_csv("../results/xgb_ens_new_csv_fold5_single_best.gz", index=False, compression='gzip')
# print(sample_submission.head())
# print('save done')


[0]	train-logloss:0.649396	valid-logloss:0.649342
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[215]	train-logloss:0.07596	valid-logloss:0.078432

0
0.0739614323552 0.0784515684788
[0]	train-logloss:0.645633	valid-logloss:0.645664
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[242]	train-logloss:0.015508	valid-logloss:0.021667

1
0.0143604188724 0.0217794412932
[0]	train-logloss:0.646907	valid-logloss:0.646877
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[271]	train-logloss:0.035238	valid-logloss:0.041563

2
0.0337505900059 0.0416431254369
[0]	train-logloss:0.644776	valid-logloss:0.64484
Multiple e